In [1]:
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [4]:
import mlflow

# Set tracking uri
mlflow.set_tracking_uri('sqlite:///mlflow.db')

# Set experiment (create if it doesn't exist)
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
# You can define a new run to keep track
with mlflow.start_run():
    mlflow.set_tag("developer", "Daniel")
    mlflow.log_param("train_path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val_path", "./data/green_tripdata_2021-02.parquet")
    
    alpha = 0.001
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    # Keep track of performance metrics
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    # Save model to MLFlow
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle/")

In [12]:
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

## Training using XGBoost and Hyperopt

In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [14]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(params, train, num_boost_round=1000, early_stopping_rounds=50, evals=[(valid, 'validation')])
        
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [15]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[17:18:57] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:9.99168                           
[1]	validation-rmse:7.41340                           
[2]	validation-rmse:6.87286                           
[3]	validation-rmse:6.73370                           
[4]	validation-rmse:6.69134                           
[5]	validation-rmse:6.66909                           
[6]	validation-rmse:6.66336                           
[7]	validation-rmse:6.65613                           
[8]	validation-rmse:6.64732                           
[9]	validation-rmse:6.63846                           
[10]	validation-rmse:6.63330                          
[11]	validation-rmse:6.63047                          
[12]	validation-rmse:6.62673                          
[13]	validation-rmse:6.62312                          
[14]	validation-rmse:6.61734                          
[15]	validation-rmse:6.61385                          
[

KeyboardInterrupt: 

In [16]:
# Train best model
params = {
    'learning_rate': 0.20472169880371677,
    'max_depth': 17,
    'min_child_weight': 1.2402611720043835,
    'objective': 'reg:linear',
    'reg_alpha': 0.28567896734700793,
    'reg_lambda': 0.004264404814393709,
    'seed': 42
}

mlflow.xgboost.autolog()

best_booster = xgb.train(params, train, num_boost_round=1000, early_stopping_rounds=50, evals=[(valid, 'validation')])

2022/05/30 17:57:13 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '24b6dbb6ea834207be0b6b71cb891f9b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[17:57:13] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.57670
[1]	validation-rmse:14.77950
[2]	validation-rmse:12.64199
[3]	validation-rmse:11.02820
[4]	validation-rmse:9.82436
[5]	validation-rmse:8.93793
[6]	validation-rmse:8.29597
[7]	validation-rmse:7.82922
[8]	validation-rmse:7.49422
[9]	validation-rmse:7.25124
[10]	validation-rmse:7.07616
[11]	validation-rmse:6.94884
[12]	validation-rmse:6.85678
[13]	validation-rmse:6.78789
[14]	validation-rmse:6.73514
[15]	validation-rmse:6.69523
[16]	validation-rmse:6.66330
[17]	validation-rmse:6.64037
[18]	validation-rmse:6.62226
[19]	validation-rmse:6.60733
[20]	validation-rmse:6.59614
[21]	validation-rmse:6.58815
[22]	validation-rmse:6.58247
[23]	validation-rmse:6.57662
[24]	validation-rmse:6.57243
[25]	validation-rmse:6.56791
[26]	validation-rmse:6.56230
[27]	validation-rmse:6.55841
[28]	validation-rmse:6.55557
[29]	validation-rmse:6.55308
[30]	validation-r

2022/05/30 18:00:25 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/05/30 18:00:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/daniel.montilla/.cache/pypoetry/virtualenvs/mlops-project-l6gstjDc-py3.9/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
